<div style="align-items: center; justify-content: space-between;">
   <img src=""  align="right" alt="Semantix-logo" width="15%">

   <h1>Data Pipeline | AWS - Whatsapp - Telegram</h1>
   <b> por <a href="https://www.linkedin.com/in/fernandohcarneiro/">Fernando Carneiro</a> </b>
</div>

---

## Sumário
1. [**Introdução**](#intro)


<a id='intro'></a>
## 1. Introdução

### 1.1 Objetivo
O objetivo deste projeto é demonstrar como é feita a extração de dados provenientes do Telegram e do WhatsApp, transferir esses dados para um ***Datalake***, realizar o processamento dos dados em lote na nuvem e então fazer a análise dos dados tratados. Esses dados fornecem a base para extrair informações valiosas, abrindo possibilidades para aprimorar serviços e explorar oportunidades de monetização.

### 1.2 O que é Pipeline?
No contexto de ciência de dados, um ***pipeline*** refere-se a uma **sequência de processos automatizados** que são encadeados para realizar tarefas específicas, desde a coleta de dados até a entrega de resultados. Esse conceito é inspirado no campo da engenharia de software, onde um pipeline representa um fluxo contínuo de desenvolvimento e entrega de software.

    <Imagem aqui>

*fonte: https://www.stitchdata.com/resources/what-is-data-pipeline/*

### 1.3 Pipeline do projeto
O projeto conta com